# Homework #3: PCA/Hyperparameter/CV
Data source: http://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data

In [1]:
from scipy.io import arff
import pandas as pd
import numpy as np

In [2]:
data = arff.loadarff('../data/4year.arff')
df = pd.DataFrame(data[0])

In [3]:
df['bankruptcy'] = (df['class']==b'1')
df.drop(columns=['class'], inplace=True)
df.columns = ['X{0:02d}'.format(k) for k in range(1,65)] + ['bankruptcy']

In [4]:
df.describe()

,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,...,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64
count,9791.000000,9791.000000,9791.000000,9749.000000,9.771000e+03,9791.000000,9791.000000,9773.000000,9792.000000,9791.000000,...,9.792000e+03,9771.000000,9791.000000,9776.000000,9791.000000,9178.000000,9760.000000,9.771000e+03,9749.000000,9561.000000
mean,0.043019,0.596404,0.130959,8.136600,6.465164e+01,-0.059273,0.059446,19.884016,1.882296,0.389040,...,7.686330e+03,-0.992263,0.035022,1.133287,0.856053,118.156064,25.194430,2.015157e+03,8.660813,35.949619
std,0.359321,4.587122,4.559074,290.647281,1.475939e+04,6.812754,0.533344,698.697015,17.674650,4.590299,...,7.605261e+04,77.007971,8.945365,8.038201,26.393305,3230.316692,1099.260821,1.171461e+05,60.838202,483.318623
min,-12.458000,0.000000,-445.910000,-0.045319,-3.794600e+05,-486.820000,-12.458000,-1.848200,-0.032371,-445.910000,...,-7.132200e+05,-7522.100000,-597.420000,-30.892000,-284.380000,0.000000,-12.656000,-1.496500e+04,-0.024390,-0.000015
25%,0.001321,0.263145,0.020377,1.047000,-5.121700e+01,-0.000578,0.003004,0.428300,1.006675,0.294440,...,2.184000e+01,0.003121,0.008768,0.885722,0.000000,5.356325,4.267700,4.323400e+01,2.938800,2.012900
50%,0.041364,0.467740,0.199290,1.591800,-5.557600e-02,0.000000,0.048820,1.088700,1.161300,0.510450,...,9.503300e+02,0.043679,0.098026,0.958305,0.002129,9.482000,6.283550,7.472900e+01,4.848900,4.041600
75%,0.111130,0.689255,0.410670,2.880400,5.573200e+01,0.065322,0.126940,2.691000,1.970225,0.714290,...,4.694550e+03,0.117170,0.242680,0.996163,0.211790,19.506000,9.938200,1.233450e+02,8.363800,9.413500
max,20.482000,446.910000,22.769000,27146.000000,1.034100e+06,322.200000,38.618000,53209.000000,1704.800000,12.602000,...,6.123700e+06,112.020000,226.760000,668.750000,1661.000000,251570.000000,108000.000000,1.077900e+07,5662.400000,21153.000000


In [5]:
sum(df.bankruptcy == True)

515

In [6]:
df.fillna(df.mean(), inplace=True)
df.isna().sum()
X_imp = df.values

In [7]:
from sklearn.model_selection import train_test_split

X, y = X_imp[:, :-1], X_imp[:, -1].astype(int)

X_train, X_test, y_train, y_test =\
    train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [8]:
import sklearn.preprocessing as skpre

stdsc = skpre.StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
print(X_train_std.shape)
X_test_std = stdsc.transform(X_test)
print(X_test_std.shape)

(6854, 64)
(2938, 64)


## PCA using pipeline

**1. Extract 3 features using PCA method.  
2. Using the selected features from above, we are going to apply LR / SVM / decision tree.  
3. Implement the methods using pipeline. (PML p185)**

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

In [10]:
# using logistics regression
pipe_lr = make_pipeline(PCA(n_components=3), LogisticRegression(random_state=1))
pipe_lr.fit(X_train_std, y_train)
print('Test Accuracy: %.3f' % pipe_lr.score(X_test_std, y_test))

Test Accuracy: 0.946


In [11]:
# using SVM
pipe_svm = make_pipeline(PCA(n_components=3), SVC(random_state=1))
pipe_svm.fit(X_train_std, y_train)
print('Test Accuracy: %.3f' % pipe_svm.score(X_test_std, y_test))

Test Accuracy: 0.947


In [12]:
# using decision tree
pipe_dt = make_pipeline(PCA(n_components=3), DecisionTreeClassifier(random_state=1))
pipe_dt.fit(X_train_std, y_train)
print('Test Accuracy: %.3f' % pipe_dt.score(X_test_std, y_test))

Test Accuracy: 0.900


## 5 fold Cross-Validation

In [10]:
from sklearn.model_selection import StratifiedKFold

In [14]:
# using logistics regression
kfold = StratifiedKFold(n_splits=5).split(X_train_std, y_train)
pipe_lr = make_pipeline(PCA(n_components=3), LogisticRegression(random_state=1))
for k, (train,test) in enumerate(kfold):
    pipe_lr.fit(X_train_std[train], y_train[train])
    score = pipe_lr.score(X_train_std[test], y_train[test])
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1, np.bincount(y_train[train]), score))

Fold:  1, Class dist.: [5195  288], Acc: 0.947
Fold:  2, Class dist.: [5195  288], Acc: 0.947
Fold:  3, Class dist.: [5195  288], Acc: 0.946
Fold:  4, Class dist.: [5195  288], Acc: 0.948
Fold:  5, Class dist.: [5196  288], Acc: 0.947


In [15]:
# using SVM
kfold = StratifiedKFold(n_splits=5).split(X_train_std, y_train)
pipe_svc = make_pipeline(PCA(n_components=3), SVC(random_state=1))
for k, (train,test) in enumerate(kfold):
    pipe_svc.fit(X_train_std[train], y_train[train])
    score = pipe_svc.score(X_train_std[test], y_train[test])
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1, np.bincount(y_train[train]), score))

Fold:  1, Class dist.: [5195  288], Acc: 0.947
Fold:  2, Class dist.: [5195  288], Acc: 0.947
Fold:  3, Class dist.: [5195  288], Acc: 0.947
Fold:  4, Class dist.: [5195  288], Acc: 0.947
Fold:  5, Class dist.: [5196  288], Acc: 0.947


In [16]:
# using decision tree
kfold = StratifiedKFold(n_splits=5).split(X_train_std, y_train)
pipe_dt = make_pipeline(PCA(n_components=3), DecisionTreeClassifier(random_state=1))
for k, (train,test) in enumerate(kfold):
    pipe_dt.fit(X_train_std[train], y_train[train])
    score = pipe_dt.score(X_train_std[test], y_train[test])
    print('Fold: %2d, Class dist.: %s, Acc: %.3f' % (k+1, np.bincount(y_train[train]), score))

Fold:  1, Class dist.: [5195  288], Acc: 0.900
Fold:  2, Class dist.: [5195  288], Acc: 0.902
Fold:  3, Class dist.: [5195  288], Acc: 0.900
Fold:  4, Class dist.: [5195  288], Acc: 0.903
Fold:  5, Class dist.: [5196  288], Acc: 0.895


## Grid search

**4. Use grid search for finding optimal hyperparameters. (PML p199). In the search, apply 5-fold cross-validation.**

In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
# using logistics regression
pipe_lr = make_pipeline(PCA(n_components=3), LogisticRegression(random_state=1))
pipe_lr.get_params().keys()

In [26]:
param_range = [0.001, 0.01, 0.1, 1.0, 10.0]

param_grid = [{'logisticregression__C': param_range}]

gs_lr = GridSearchCV(estimator=pipe_lr, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
gs_lr = gs_lr.fit(X_train_std, y_train)
print(gs_lr.best_score_)
print(gs_lr.best_params_)

0.9471839511891261
{'logisticregression__C': 0.001}


In [18]:
pipe_svc = make_pipeline(PCA(n_components=3), SVC(random_state=1))
pipe_svc.get_params().keys()

In [20]:
# using SVM
param_range = [0.001, 0.01, 0.1, 1.0, 10.0]

param_grid = [{'svc__C': param_range, 'svc__kernel': ['linear']},
              {'svc__C': param_range, 'svc__gamma': param_range,  'svc__kernel': ['rbf']}]

gs_svc = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
gs_svc = gs_svc.fit(X_train_std, y_train)
print(gs_svc.best_score_)
print(gs_svc.best_params_)

0.9474759219920459
{'svc__C': 0.001, 'svc__kernel': 'linear'}


In [13]:
# using decision tree
pipe_dt = make_pipeline(PCA(n_components=3), DecisionTreeClassifier(random_state=1))
pipe_dt.get_params().keys()

In [16]:
param_grid = [{'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5, 6, 7, None]}]

gs_dt = GridSearchCV(estimator=pipe_dt, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
gs_dt = gs_dt.fit(X_train_std, y_train)
print(gs_dt.best_score_)
print(gs_dt.best_params_)

0.9474759219920459
{'decisiontreeclassifier__max_depth': 1}
